In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import butcher1 as butcher
import bro
import os

from numpy import savetxt
from astropy.io import ascii
from astropy.time import Time
from scipy.optimize import curve_fit
from astropy.timeseries import LombScargle
from scipy.stats import median_absolute_deviation

from collections import deque
from bisect import insort, bisect_left
from itertools import islice

import matplotlib
plt.style.use('seaborn-dark-palette')
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

In [ ]:
def binner(bins, time, flux, eflux):
    means, errors, midpoints = [], [], []
    for i in range(len(bins)-1):
        binned_mask = (time < bins[i+1])*(time > bins[i])
        if np.all(binned_mask == False):
            pass
        else:
            midpoints.append((bins[i] + bins[i+1])/2)
            means.append(np.average(flux[binned_mask], weights = 1/eflux[binned_mask]**2))
            errors.append(np.std(flux[binned_mask])/np.sqrt(len(flux[binned_mask])))
    return means, errors, midpoints

In [ ]:
henk = np.array([True, True, False])
print(np.any(henk == True))

In [ ]:
# find a0,a1,a2,a3,a4 that is the best fit to the plot above

def ringaling(phi,*c):
    # c is a numpy array containing an odd number of coeffieicnts
    # so that c[0] + c[1]*np.sin(phi) + c[2]*np.cos(phi) + c[3]*np.sin(2*phi) + c[4]*np.cos(2*phi) + .....
    #if (c.size%2 == 0):
    #    print('whoa! we need an odd number of coefficients in c')
    #    return 1
    c = np.array(c)
    npairs = (c.size-1)/2
    result = 0
    for i in np.arange(npairs):
        a_sin = c[((i*2)+1).astype(int)]
        a_cos = c[((i*2)+2).astype(int)]
        result = result + a_sin*np.sin(phi*(i+1)) + a_cos*np.cos(phi*(i+1))
    return result+c[0]

## Normalising it all

In [ ]:
#Reading in the data

#Get the data directory
cwd = os.getcwd()
data_dir = cwd.replace('Figure_4', 'Data\\')

#ASAS data
orgasas_data = ascii.read(data_dir + 'asas.csv')
asas_mask = (orgasas_data['emag'] < 0.05)
asas_data = orgasas_data[asas_mask]

asas_flux = butcher.mag_to_flux(asas_data['mag'])
asas_eflux = butcher.emag_to_eflux(asas_data['mag'], asas_data['emag'])

#ASASSN data
orgasassn_data = ascii.read(data_dir + 'asassn.csv')
asassn_mask = (orgasassn_data['emag'] < 0.05)
asassn_data = orgasassn_data[asassn_mask]

asassn_flux = butcher.mag_to_flux(asassn_data['mag'])
asassn_eflux = butcher.emag_to_eflux(asassn_data['mag'], asassn_data['emag'])

#KELT data
orgkelt_data = ascii.read(data_dir + 'kelt.csv')
kelt_mask = (orgkelt_data['emag'] < 0.05)
kelt_data = orgkelt_data[kelt_mask]

kelt_flux = butcher.mag_to_flux(kelt_data['mag'])
kelt_eflux = butcher.emag_to_eflux(kelt_data['mag'], kelt_data['emag'])

#PROMPT data
orgprompt_data = ascii.read(data_dir + 'prompt.csv') #time is JD-2450000
prompt_mask = (orgprompt_data['emag'] < 0.05)
prompt_data = orgprompt_data[prompt_mask]

prompt_flux = butcher.mag_to_flux(prompt_data['mag'])
prompt_eflux = butcher.emag_to_eflux(prompt_data['mag'], prompt_data['emag'])

#ROAD data
orgroad_data = ascii.read(data_dir + 'road.csv') #time is JD-2450000
road_mask = (orgroad_data['emag'] < 0.05)
road_data = orgroad_data[road_mask]

road_flux = butcher.mag_to_flux(road_data['mag'])
road_eflux = butcher.emag_to_eflux(road_data['mag'], road_data['emag'])

In [ ]:
asas_flux = butcher.long_correct(asas_data['MJD'], asas_flux, asas_eflux)
asassn_flux = butcher.long_correct(asassn_data['MJD'], asassn_flux, asassn_eflux)
kelt_flux = butcher.long_correct(kelt_data['HJD'], kelt_flux, kelt_eflux)
prompt_flux = butcher.long_correct(prompt_data['HJD'], prompt_flux, prompt_eflux)
road_flux = butcher.long_correct(road_data['HJD'], road_flux, road_eflux)

## Plotting the data after long correct

In [ ]:
times = [asas_data['MJD'], asassn_data['MJD'], kelt_data['HJD'], prompt_data['HJD'], road_data['HJD']]
fluxes = [asas_flux, asassn_flux, kelt_flux, prompt_flux, road_flux]
uncertainties = [asas_eflux, asassn_eflux, kelt_eflux, prompt_eflux, road_eflux]
names = ['ASAS', 'ASAS-SN', 'KELT', 'PROMPT', 'ROAD']

fig, ax = plt.subplots(5, sharey= True)

for i in range(5):
    time, flux, eflux = times[i], fluxes[i], uncertainties[i]
    
    

    ax[i].errorbar(time, flux, eflux, fmt='.', markersize= 3, elinewidth=0.5)
    figure = plt.gcf()
    figure.set_size_inches(12, 20)
    ax[i].set_title("Uncorrected " +str(names[i]))
    #ax[i].set_ylim(0.86, 1.08)
    print("Scatter is", np.std(flux))


In [ ]:
for i in range(5):
    time, flux, eflux = times[i], fluxes[i], uncertainties[i]
    
    frequencies = 1/np.linspace(2, 10, 3000)
    power = LombScargle(time, flux-np.mean(flux), dy = eflux).power(frequencies)
    max_period = 1/frequencies[list(power).index(np.max(power))]
    
    means, errors, midpoints = binner(np.linspace(0, max_period, int(2*len(time)**(1/3.))), time%max_period, flux, eflux)
    
    popt, pcov = curve_fit(ringaling, np.array(midpoints)*2*np.pi/(max_period), means, sigma = errors, absolute_sigma = True, p0 = [1,0,0], maxfev = 5000)
    phi = np.arange(200)*2*np.pi / 200
    
    plt.errorbar(midpoints, means, errors, ms = 3, fmt = '.')
    plt.plot(phi*max_period/(2*np.pi), ringaling(phi, *popt))
    plt.show()
    
    print("The max is at: ", '{0:.3f}'.format(np.max(ringaling(phi, *popt))))
    print("The min is at: ", '{0:.3f}'.format(np.min(ringaling(phi, *popt))))
    print("The amp is at: ", '{0:.3f}'.format(np.max(ringaling(phi, *popt))-np.min(ringaling(phi, *popt))))
    print(np.sqrt(np.diag(pcov)))

## Periodic correction

In [ ]:
#Full periodogram
names = ['ASAS', 'ASASSN', 'KELT', 'PROMPT', 'ROAD']
corr_fluxes_1, corr_fluxes_2 = [], []

org_powers, powers2 = [], []
for j in range(5):
    time, flux, eflux = times[j], fluxes[j], uncertainties[j]

    road_corrflux, periods, road_freq1, road_power1 = bro.short_correct(time, flux, eflux, min_chunk_size = 10)

    #Get the uncorrected lombscargle
    frequencies = 1/np.linspace(2, 10, 3000)
    org_power = LombScargle(time, flux-np.mean(flux), dy = eflux).power(frequencies)
    org_max_period = 1/frequencies[list(org_power).index(np.max(org_power))]
    #print("The highest power period is ", '{0:.3f}'.format(org_max_period), ' days.')

    #Get the butcher corrected lombscargle
    butcher_flux, butcher_periods = butcher.short_correct(time, flux, eflux)
    frequencies = 1/np.linspace(2, 10, 3000)
    butcher_power = LombScargle(time, butcher_flux-np.mean(butcher_flux), dy = eflux).power(frequencies)
    butcher_max_period = 1/frequencies[list(butcher_power).index(np.max(butcher_power))]
    #print("The highest power period is ", '{0:.3f}'.format(butcher_max_period), ' days.')
    
    #Get the bro corrected lombscargle
    frequencies = 1/np.linspace(2, 10, 3000)
    power = LombScargle(time, road_corrflux-np.mean(road_corrflux), dy = eflux).power(frequencies)
    max_period = 1/frequencies[list(power).index(np.max(power))]
    #print("The highest power period is ", '{0:.3f}'.format(max_period), ' days.')
    
    road_corrflux2, periods2, road_freq1, road_power1 = bro.short_correct(time, road_corrflux, eflux, min_chunk_size = 10)
    frequencies = 1/np.linspace(2, 10, 3000)
    power2 = LombScargle(time, road_corrflux2-np.mean(road_corrflux2), dy = eflux).power(frequencies)
    max_period = 1/frequencies[list(power2).index(np.max(power2))]
    
    org_powers.append(org_power)
    powers2.append(power2)
    
    
    #plt.plot(1/frequencies, power2-power)
    #fig = plt.gcf()
    #fig.set_size_inches(12,9)
    #plt.show()
    
    corr_fluxes_1.append(road_corrflux)
    corr_fluxes_2.append(road_corrflux2)
    
    print(periods)
    print(periods2)
    print("The scatter is now: ", np.std(road_corrflux))
    print("The scatter is now: ", np.std(road_corrflux2))
#0.04687787284474136
#0.021
#0.028
#0.069
#0.023

In [ ]:
fig, ax = plt.subplots(5)
for i in range(5):
    ax[i].plot(1/frequencies, org_powers[i], alpha = 0.7, c='grey', label = 'Pre-Correction ' + names[i])
    #ax.plot(1/frequencies, butcher_power, alpha = 0.7, c = 'orange', label = 'Butcher-Correction')
    ax[i].plot(1/frequencies, powers2[i], alpha = 0.7,label = 'Post-Correction ' + names[i])
    ax[i].legend(fontsize = 18)

fig.text(0.35, 0.09, 'Signal Period (Days)', fontsize = 24)
fig.text(0.02, 0.35, 'Signal Power (Arbitrary Units)', rotation = 90, fontsize = 24)
fig = plt.gcf()
fig.set_size_inches(12,20)
#plt.savefig('Removing_Dominant_Cycle.pdf')
plt.show()